In [1]:
### Import libraries

import igraph as ig
import pandas as pd
import numpy as np

In [2]:
# Load data
file_name = "../data/IOT-AL-universal.csv"
acc_log = pd.read_csv(file_name)
acc_log = acc_log[acc_log.columns[1:]]
acc_log["rname"] = acc_log["rname"] + max(acc_log.uname.unique()) + 1

print("Done!")
acc_log.head(5)

Done!


,role,age,health,uname,type,area,mode,temperature,lockstatus,rname,location,timeday,op,ACTION
0,child,3,health,0,Smart locks,livingroom,armed,-10,locked,123,livingroom,day,access,1
1,child,3,health,0,Smart locks,livingroom,armed,-10,locked,123,livingroom,day,control,1
2,child,3,health,0,Smart locks,livingroom,armed,-10,locked,123,livingroom,day,arm,1
3,child,3,health,0,Smart locks,livingroom,armed,-10,locked,123,livingroom,midday,access,1
4,child,3,health,0,Smart locks,livingroom,armed,-10,locked,123,livingroom,midday,control,1


In [3]:
user_attributes = ["role", "age", "health", "uname"]
users = acc_log[user_attributes].drop_duplicates()
users = users.reset_index(drop=True)
print("|U| =", len(users))
users.head()

|U| = 120


,role,age,health,uname
0,child,3,health,0
1,child,3,anormal,1
2,child,3,emergency,2
3,child,5,health,3
4,child,5,anormal,4


In [4]:
res_attributes = ["area", "mode", "temperature", "lockstatus", "rname"]
resrs = acc_log[res_attributes].drop_duplicates()
resrs = resrs.reset_index(drop=True)
print("|R| =", len(resrs))
resrs.head()

|R| = 990


,area,mode,temperature,lockstatus,rname
0,livingroom,armed,-10,locked,123
1,livingroom,armed,-10,unlocked,124
2,livingroom,armed,-5,locked,125
3,livingroom,armed,-5,unlocked,126
4,livingroom,armed,0,locked,127


In [5]:
edges_attributes = ["uname", "rname", "location", "timeday", "op", "ACTION"]
edges = acc_log[edges_attributes].drop_duplicates()
edges = edges.reset_index(drop=True)
edges["weight"] = np.round(np.random.random(len(edges)), 2)
print("|E| =", len(edges))
edges.head()

|E| = 5702400


,uname,rname,location,timeday,op,ACTION,weight
0,0,123,livingroom,day,access,1,0.17
1,0,123,livingroom,day,control,1,0.05
2,0,123,livingroom,day,arm,1,0.54
3,0,123,livingroom,midday,access,1,0.61
4,0,123,livingroom,midday,control,1,0.16


In [6]:
### Create a graph

# Iteration over tuples in the dataframe
tuple_list = edges.itertuples(index=False)

# Using the TupleList method to build the network
bip_network = ig.Graph.TupleList(tuple_list, directed=False,
                                 edge_attrs=edges.columns[2:].tolist())


print(bip_network.summary())

# Check if it is a bipartite network
if bip_network.is_bipartite():
    print("It is bipartite!")
else:
    print("The network is not bipartite.")

IGRAPH UNW- 1110 5702400 -- 
+ attr: name (v), ACTION (e), location (e), op (e), timeday (e), weight (e)
It is bipartite


In [26]:
### Add user attributes

# Add type of node:     0=User      1=Resource
user_nodes = bip_network.vs.select(name_le=max(users.uname.unique()))
resource_nodes = bip_network.vs.select(name_gt=max(users.uname.unique()))
user_nodes["type"] = 0
resource_nodes["type"] = 1

# Add attributes
for attr in user_attributes[:-1]:   # User attributes
    user_nodes[attr] = users[attr]
for attr in res_attributes[:-1]:    # Resource attributes
    resource_nodes[attr] = resrs[attr]

# Remove objecto to free memory
del user_nodes, resource_nodes

In [78]:
# Save the graph
bip_network.write("iot_bip_graph.graphml")